## CNN(Convolutional Neural Network)

### 내용
1. CNN개념
    - CNN이란 - CNN(Convolutional Neural Network)은 이미지 처리와 같은 시각적 데이터를 다루기 위해 설계된 신경망 구조
        - 전통적인 신경망보다 적은 파라미터로 이미지의 공간적 정보를 효과적으로 처리
    - CNN의 기본 원리
        - Convolution 연산: 이미지의 일부 특징을 추출
        - Pooling 연산: 특징 맵의 크기를 줄이고, 계산 비용을 감소시키며, 중요한 특징을 강조
        - Convolutional Layer: 필터(커널)를 사용하여 특징 맵 생성
        - Pooling Layer: 다운샘플링을 통해 특징 맵 크기 축소(Max Pooling)
        - Fully Connected Layer: 분류기 역할 수행

2. PyTorch에서 CNN 구현
    - torch.nn 모듈을 사용하여 CNN 레이어 구현
        - nn.Conv2d, nn.MaxPool2d, nn.ReLU
    

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

3. CNN 모델 학습
    - MNIST 데이터셋

In [5]:
# 데이터 전처리 변환
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Failed to download (trying next):
<urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>



100%|██████████| 9.91M/9.91M [00:01<00:00, 6.25MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>



100%|██████████| 28.9k/28.9k [00:00<00:00, 152kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>



100%|██████████| 1.65M/1.65M [00:00<00:00, 1.68MB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>



100%|██████████| 4.54k/4.54k [00:00<00:00, 565kB/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



- 모델 학습 및 평가
    - 손실함수와 옵티마이저 설정
    - 학습 루프 작성

In [6]:
# 모델, 손실 함수, 옵티마이저 설정
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Epoch 1, Loss: 0.13110344316891084
Epoch 2, Loss: 0.04326649268244703
Epoch 3, Loss: 0.02932308912035854
Epoch 4, Loss: 0.02159774273807852
Epoch 5, Loss: 0.01690759526417581
Accuracy: 99.2%


- 5분 1초 소요

##### GPU로 변경

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=4)

In [9]:
# 모델, 손실 함수, 옵티마이저 설정
model = SimpleCNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# 학습 루프
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Epoch 1, Loss: 0.12831695611067118
Epoch 2, Loss: 0.041017322327631216
Epoch 3, Loss: 0.028083588227666528
Epoch 4, Loss: 0.01953138000596858
Epoch 5, Loss: 0.01632714946325553
Accuracy: 99.14%


- GPU(CUDA) 사용시 1분 6.7초 소요 / $ \frac {1} {5} $ 시간 절약